# Sudoku-5 : Particle Swarm Optimization (Python)

**Niveau** : Metaheuristique | **Duree** : ~20 min | **Prerequis** : Sudoku-0 Environment

## Navigation

| << Precedent | [Index](README.md) | Suivant >> |
|-------------|---------------------|-----------|
| [Sudoku-4-SimulatedAnnealing-Python](Sudoku-4-SimulatedAnnealing-Python.ipynb) | | [Sudoku-6-AIMA-CSP-Python](Sudoku-6-AIMA-CSP-Python.ipynb) |

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre les principes de l'optimisation par essaim de particules (PSO)
2. Adapter le PSO a la resolution de Sudoku
3. Implementer un solveur PSO avec la bibliotheque `mealpy`
4. Comparer les performances avec les autres metaheuristiques

---

Ce notebook implemente un solveur de Sudoku utilisant le Particle Swarm Optimization (PSO) en Python.
C'est l'equivalent Python du notebook C# `Sudoku-5-PSO-Csharp.ipynb`.

## Introduction

Le **Particle Swarm Optimization** (PSO) est une metaheuristique inspiree du comportement social des oiseaux et des poissons. Developpe par Kennedy et Eberhart en 1995, l'algorithme simule un essaim de particules qui explorent l'espace de recherche.

### Principes fondamentaux

1. **Particules** : Chaque particule represente une solution candidate
2. **Position** : La position de la particule = configuration de la grille
3. **Vitesse** : Direction et vitesse de deplacement dans l'espace de recherche
4. **pBest** : Meilleure position personnelle de la particule
5. **gBest** : Meilleure position globale de l'essaim

### Equation de mise a jour

$$v(t+1) = w \cdot v(t) + c_1 \cdot r_1 \cdot (pBest - x(t)) + c_2 \cdot r_2 \cdot (gBest - x(t))$$
$$x(t+1) = x(t) + v(t+1)$$

Ou :
- $w$ : poids d'inertie (impact de la vitesse precedente)
- $c_1$, $c_2$ : coefficients d'apprentissage (cognitif et social)
- $r_1$, $r_2$ : nombres aleatoires [0, 1]

## Installation

```bash
pip install mealpy numpy matplotlib
```

In [1]:
# Imports
import numpy as np
import time
import random
from typing import List, Tuple, Optional
import matplotlib.pyplot as plt

# mealpy est optionnel - ce notebook implemente PSO manuellement
try:
    from mealpy import PSO
    from mealpy.problem import Problem
    MEALPY_AVAILABLE = True
    print("mealpy importe avec succes")
except ImportError:
    MEALPY_AVAILABLE = False
    print("mealpy non installe (optionnel) - ce notebook utilise PSO manuel")

mealpy non installe (optionnel) - ce notebook utilise PSO manuel


In [2]:
# Configuration du chemin vers les puzzles
import os
from pathlib import Path

NOTEBOOK_DIR = Path(r"D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku")
PUZZLES_DIR = NOTEBOOK_DIR / "Puzzles"

if PUZZLES_DIR.exists():
    print(f"Dossier Puzzles: {PUZZLES_DIR}")
else:
    print(f"ATTENTION: Dossier Puzzles non trouve a {PUZZLES_DIR}")
    PUZZLES_DIR = Path(os.getcwd()) / "Puzzles"

Dossier Puzzles: D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku\Puzzles


## 1. Classe SudokuGrid

Representation d'une grille de Sudoku avec methodes pour calculer le nombre d'erreurs.

In [3]:
class SudokuGrid:
    """Representation d'une grille de Sudoku 9x9."""
    
    def __init__(self, grid: Optional[np.ndarray] = None):
        if grid is None:
            self.cells = np.zeros((9, 9), dtype=int)
        else:
            self.cells = grid.copy()
    
    @classmethod
    def from_string(cls, s: str) -> 'SudokuGrid':
        s = s.replace('.', '0').replace(' ', '').replace('\n', '')
        if len(s) != 81:
            raise ValueError(f"La chaine doit avoir 81 caracteres")
        grid = cls()
        grid.cells = np.array([int(c) for c in s], dtype=int).reshape(9, 9)
        return grid
    
    def clone(self) -> 'SudokuGrid':
        return SudokuGrid(self.cells.copy())
    
    def count_errors(self) -> int:
        """Compte le nombre total d'erreurs (doublons)."""
        errors = 0
        
        # Erreurs par ligne
        for i in range(9):
            row = self.cells[i, :]
            row_nonzero = row[row > 0]
            errors += len(row_nonzero) - len(np.unique(row_nonzero))
        
        # Erreurs par colonne
        for j in range(9):
            col = self.cells[:, j]
            col_nonzero = col[col > 0]
            errors += len(col_nonzero) - len(np.unique(col_nonzero))
        
        # Erreurs par bloc 3x3
        for box_row in range(3):
            for box_col in range(3):
                block = self.cells[box_row*3:(box_row+1)*3, box_col*3:(box_col+1)*3].flatten()
                block_nonzero = block[block > 0]
                errors += len(block_nonzero) - len(np.unique(block_nonzero))
        
        return errors
    
    def is_solved(self) -> bool:
        """Verifie si la grille est resolue."""
        if np.any(self.cells == 0):
            return False
        return self.count_errors() == 0
    
    def __str__(self) -> str:
        lines = []
        for r in range(9):
            if r > 0 and r % 3 == 0:
                lines.append('-' * 21)
            row_str = ''
            for c in range(9):
                if c > 0 and c % 3 == 0:
                    row_str += '| '
                val = self.cells[r, c]
                row_str += (str(val) if val != 0 else '.') + ' '
            lines.append(row_str)
        return '\n'.join(lines)

def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

# Charger les puzzles
easy_puzzles = load_puzzles(str(PUZZLES_DIR / 'Sudoku_Easy51.txt'), max_puzzles=5)
print(f"Puzzles charges: {len(easy_puzzles)}")

test_grid = SudokuGrid.from_string(easy_puzzles[0])
print("\nGrille de test:")
print(test_grid)

Puzzles charges: 5

Grille de test:
9 . 2 | . . 5 | 4 . 3 
1 . . | . 6 3 | . 2 5 
5 . 8 | 4 . 7 | . 6 . 
---------------------
. 2 6 | 3 . 9 | . . 1 
. 5 7 | . 1 . | 2 9 . 
. 9 . | 6 7 . | 5 3 . 
---------------------
2 4 . | 5 3 . | 6 . . 
7 . 5 | 2 . . | 3 . 4 
. 8 . | . 4 1 | 9 5 . 


## 2. Approches PSO pour Sudoku

Il existe plusieurs facons d'encoder un Sudoku pour PSO. Nous allons explorer deux approches :

### Approche 1 : Encodage direct par cellules
Chaque particule contient 81 valeurs (1-9) pour chaque cellule.

### Approche 2 : Encodage par permutations de blocs
Chaque bloc 3x3 est initialise avec les valeurs correctes, puis PSO melange les permutations.

## 3. Implementation manuelle du PSO

Nous implementons un PSO specifique au Sudoku, sans utiliser mealpy, pour mieux comprendre l'algorithme.

In [4]:
class MatrixHelperPSO:
    """Helper pour la manipulation des matrices Sudoku."""
    
    SIZE = 9
    BLOCK_SIZE = 3
    
    @staticmethod
    def corner(block: int) -> Tuple[int, int]:
        """Retourne le coin superieur gauche d'un bloc."""
        r = (block // 3) * 3
        c = (block % 3) * 3
        return (r, c)
    
    @staticmethod
    def random_matrix(rng: random.Random, problem: np.ndarray) -> np.ndarray:
        """Cree une matrice aleatoire valide par bloc."""
        result = problem.copy()
        
        for block in range(9):
            corner_r, corner_c = MatrixHelperPSO.corner(block)
            values = list(range(1, 10))
            
            # Retirer les valeurs deja presentes
            for r in range(corner_r, corner_r + 3):
                for c in range(corner_c, corner_c + 3):
                    val = problem[r, c]
                    if val != 0 and val in values:
                        values.remove(val)
            
            # Melanger
            rng.shuffle(values)
            
            # Remplir les cellules vides
            pointer = 0
            for r in range(corner_r, corner_r + 3):
                for c in range(corner_c, corner_c + 3):
                    if result[r, c] == 0:
                        result[r, c] = values[pointer]
                        pointer += 1
        
        return result
    
    @staticmethod
    def neighbor_matrix(rng: random.Random, problem: np.ndarray, matrix: np.ndarray) -> np.ndarray:
        """Genere un voisin par echange dans un bloc."""
        result = matrix.copy()
        
        block = rng.randint(0, 8)
        corner_r, corner_c = MatrixHelperPSO.corner(block)
        
        cells = []
        for r in range(corner_r, corner_r + 3):
            for c in range(corner_c, corner_c + 3):
                if problem[r, c] == 0:
                    cells.append((r, c))
        
        if len(cells) < 2:
            return result
        
        k1 = rng.randint(0, len(cells) - 1)
        k2 = (k1 + rng.randint(1, len(cells) - 1)) % len(cells)
        
        r1, c1 = cells[k1]
        r2, c2 = cells[k2]
        
        result[r1, c1], result[r2, c2] = result[r2, c2], result[r1, c1]
        
        return result
    
    @staticmethod
    def merge_matrices(rng: random.Random, m1: np.ndarray, m2: np.ndarray) -> np.ndarray:
        """Fusionne deux matrices par blocs."""
        result = m1.copy()
        
        for block in range(9):
            if rng.random() < 0.50:
                corner_r, corner_c = MatrixHelperPSO.corner(block)
                for r in range(corner_r, corner_r + 3):
                    for c in range(corner_c, corner_c + 3):
                        result[r, c] = m2[r, c]
        
        return result

print("MatrixHelperPSO defini.")

MatrixHelperPSO defini.


In [5]:
class SudokuPSO:
    """Representation d'une grille Sudoku avec calcul d'erreur."""
    
    def __init__(self, cell_values: np.ndarray):
        self.cell_values = cell_values.copy()
    
    @staticmethod
    def new(cell_values: np.ndarray) -> 'SudokuPSO':
        return SudokuPSO(cell_values.copy())
    
    @property
    def error(self) -> int:
        """Compte les erreurs (valeurs manquantes dans lignes/colonnes/blocs)."""
        return self._count_errors(True) + self._count_errors(False)
    
    def _count_errors(self, count_by_row: bool) -> int:
        errors = 0
        for i in range(9):
            counts = np.zeros(9, dtype=int)
            for j in range(9):
                cell_value = self.cell_values[i, j] if count_by_row else self.cell_values[j, i]
                if cell_value > 0:
                    counts[cell_value - 1] += 1
            
            for k in range(9):
                if counts[k] == 0:
                    errors += 1
        return errors

print("SudokuPSO defini.")

SudokuPSO defini.


In [6]:
from enum import Enum

class OrganismType(Enum):
    WORKER = 1
    EXPLORER = 2

class Organism:
    """Representation d'une particule (organisme)."""
    
    def __init__(self, org_type: OrganismType, matrix: np.ndarray, error: int, age: int = 0):
        self.type = org_type
        self.matrix = matrix.copy()
        self.error = error
        self.age = age

print("Classes OrganismType et Organism definies.")

Classes OrganismType et Organism definies.


## 4. Solveur PSO

In [7]:
import math

class PSOSudokuSolver:
    """Solveur de Sudoku par PSO."""
    
    def __init__(self, num_organisms: int = 200, max_epochs: int = 5000, 
                 max_restarts: int = 20, worker_ratio: float = 0.90,
                 max_age: int = 1000, mutation_rate: float = 0.001):
        self.num_organisms = num_organisms
        self.max_epochs = max_epochs
        self.max_restarts = max_restarts
        self.worker_ratio = worker_ratio
        self.max_age = max_age
        self.mutation_rate = mutation_rate
        self.energy_history = []
    
    def solve(self, puzzle: SudokuGrid) -> Tuple[SudokuGrid, bool]:
        sudoku = SudokuPSO(puzzle.cells)
        best_error = float('inf')
        best_solution = None
        
        for attempt in range(self.max_restarts):
            print(f"Tentative {attempt + 1}/{self.max_restarts}")
            rng = random.Random(attempt)
            solution = self._solve_internal(sudoku, rng)
            
            if solution.error < best_error:
                best_error = solution.error
                best_solution = solution
            
            if solution.error == 0:
                return SudokuGrid(solution.cell_values), True
        
        return SudokuGrid(best_solution.cell_values), best_error == 0
    
    def _solve_internal(self, sudoku: SudokuPSO, rng: random.Random) -> SudokuPSO:
        num_workers = int(self.num_organisms * self.worker_ratio)
        hive = []
        
        best_error = float('inf')
        best_solution = None
        
        # Initialisation de l'essaim
        for i in range(self.num_organisms):
            org_type = OrganismType.WORKER if i < num_workers else OrganismType.EXPLORER
            
            random_matrix = MatrixHelperPSO.random_matrix(rng, sudoku.cell_values)
            random_sudoku = SudokuPSO.new(random_matrix)
            err = random_sudoku.error
            
            org = Organism(org_type, random_matrix, err, 0)
            hive.append(org)
            
            if err < best_error:
                best_error = err
                best_solution = random_sudoku
        
        epoch = 0
        while epoch < self.max_epochs:
            if epoch % 1000 == 0:
                print(f"  Epoch {epoch}, Best error: {best_error}")
                self.energy_history.append(best_error)
            
            if best_error == 0:
                break
            
            # Mise a jour de chaque organisme
            for i in range(self.num_organisms):
                if hive[i].type == OrganismType.WORKER:
                    # Les workers explorent leur voisinage
                    neighbor = MatrixHelperPSO.neighbor_matrix(
                        rng, sudoku.cell_values, hive[i].matrix
                    )
                    neighbor_sudoku = SudokuPSO.new(neighbor)
                    neighbor_error = neighbor_sudoku.error
                    
                    p = rng.random()
                    if neighbor_error < hive[i].error or p < self.mutation_rate:
                        hive[i].matrix = neighbor.copy()
                        if neighbor_error < hive[i].error:
                            hive[i].age = 0
                        hive[i].error = neighbor_error
                        
                        if neighbor_error < best_error:
                            best_error = neighbor_error
                            best_solution = neighbor_sudoku
                    else:
                        hive[i].age += 1
                        if hive[i].age > self.max_age:
                            # Reinitialiser
                            random_matrix = MatrixHelperPSO.random_matrix(rng, sudoku.cell_values)
                            random_sudoku = SudokuPSO.new(random_matrix)
                            hive[i] = Organism(OrganismType.WORKER, random_matrix, random_sudoku.error, 0)
                else:
                    # Les explorers cherchent aleatoirement
                    random_matrix = MatrixHelperPSO.random_matrix(rng, sudoku.cell_values)
                    random_sudoku = SudokuPSO.new(random_matrix)
                    hive[i].matrix = random_matrix.copy()
                    hive[i].error = random_sudoku.error
                    
                    if hive[i].error < best_error:
                        best_error = hive[i].error
                        best_solution = random_sudoku
            
            # Fusion du meilleur worker avec le meilleur explorer
            workers = hive[:num_workers]
            explorers = hive[num_workers:]
            
            best_worker = min(workers, key=lambda o: o.error)
            best_explorer = min(explorers, key=lambda o: o.error)
            worst_worker = max(workers, key=lambda o: o.error)
            
            merged = MatrixHelperPSO.merge_matrices(rng, best_worker.matrix, best_explorer.matrix)
            merged_sudoku = SudokuPSO.new(merged)
            
            worst_worker_idx = workers.index(worst_worker)
            hive[worst_worker_idx] = Organism(OrganismType.WORKER, merged, merged_sudoku.error, 0)
            
            if merged_sudoku.error < best_error:
                best_error = merged_sudoku.error
                best_solution = merged_sudoku
            
            epoch += 1
        
        return best_solution

print("PSOSudokuSolver defini.")

PSOSudokuSolver defini.


## 5. Test sur un Puzzle Facile

In [8]:
print("=== Test : Puzzle Facile ===")
puzzle = SudokuGrid.from_string(easy_puzzles[0])
print("Puzzle original:")
print(puzzle)

solver = PSOSudokuSolver(
    num_organisms=200,
    max_epochs=3000,
    max_restarts=10
)

start = time.time()
result, solved = solver.solve(puzzle)
elapsed = time.time() - start

print(f"\nSolution trouvee en {elapsed:.2f}s:")
print(result)
print(f"\nSolution valide: {solved}")

=== Test : Puzzle Facile ===
Puzzle original:
9 . 2 | . . 5 | 4 . 3 
1 . . | . 6 3 | . 2 5 
5 . 8 | 4 . 7 | . 6 . 
---------------------
. 2 6 | 3 . 9 | . . 1 
. 5 7 | . 1 . | 2 9 . 
. 9 . | 6 7 . | 5 3 . 
---------------------
2 4 . | 5 3 . | 6 . . 
7 . 5 | 2 . . | 3 . 4 
. 8 . | . 4 1 | 9 5 . 
Tentative 1/10
  Epoch 0, Best error: 23



Solution trouvee en 1.85s:
9 6 2 | 1 8 5 | 4 7 3 
1 7 4 | 9 6 3 | 8 2 5 
5 3 8 | 4 2 7 | 1 6 9 
---------------------
8 2 6 | 3 5 9 | 7 4 1 
3 5 7 | 8 1 4 | 2 9 6 
4 9 1 | 6 7 2 | 5 3 8 
---------------------
2 4 9 | 5 3 8 | 6 1 7 
7 1 5 | 2 9 6 | 3 8 4 
6 8 3 | 7 4 1 | 9 5 2 

Solution valide: True


### Interpretation : Premier test

| Aspect | Observation |
|--------|-------------|
| Resultat | Le PSO peut trouver la solution pour les puzzles faciles |
| Temps | Variable selon la configuration |
| Workers vs Explorers | Les workers exploitent, les explorers decouvrent |

> **Point cle** : Le PSO combine l'exploitation (convergence vers gBest) avec l'exploration (recherche aleatoire).

## 6. Benchmark sur Plusieurs Puzzles

In [9]:
def benchmark_pso(puzzles: List[str], num_puzzles: int = 5):
    """Benchmark du PSO sur plusieurs puzzles."""
    print(f"\n=== Benchmark: PSO ({num_puzzles} puzzles) ===")
    
    results = []
    total_time = 0
    solved_count = 0
    
    for i, puzzle_str in enumerate(puzzles[:num_puzzles]):
        grid = SudokuGrid.from_string(puzzle_str)
        empty_count = np.sum(grid.cells == 0)
        
        solver = PSOSudokuSolver(
            num_organisms=200,
            max_epochs=3000,
            max_restarts=10
        )
        
        start = time.time()
        result, solved = solver.solve(grid)
        elapsed = time.time() - start
        
        errors = result.count_errors()
        total_time += elapsed
        if solved:
            solved_count += 1
        
        status = "OK" if solved else f"{errors} erreurs"
        print(f"  Puzzle {i+1}: {status}, {empty_count} vides, {elapsed:.2f}s")
        results.append({'solved': solved, 'errors': errors, 'time': elapsed})
    
    print(f"\nResume:")
    print(f"  Resolus: {solved_count}/{num_puzzles}")
    print(f"  Temps total: {total_time:.2f}s")
    print(f"  Temps moyen: {total_time/num_puzzles:.2f}s")
    
    return results

benchmark_pso(easy_puzzles, num_puzzles=3)


=== Benchmark: PSO (3 puzzles) ===
Tentative 1/10
  Epoch 0, Best error: 23


  Puzzle 1: OK, 36 vides, 1.86s
Tentative 1/10
  Epoch 0, Best error: 29


  Epoch 1000, Best error: 2


  Epoch 2000, Best error: 2


Tentative 2/10
  Epoch 0, Best error: 29


...

  Epoch 1000, Best error: 2


  Epoch 2000, Best error: 2


Tentative 10/10
  Epoch 0, Best error: 28


  Puzzle 3: OK, 51 vides, 513.81s

Resume:
  Resolus: 3/3
  Temps total: 650.83s
  Temps moyen: 216.94s


[{'solved': True, 'errors': 0, 'time': 1.8572096824645996},
 {'solved': True, 'errors': 0, 'time': 135.15540671348572},
 {'solved': True, 'errors': 0, 'time': 513.8140983581543}]

## 7. Comparaison des Parametres

In [10]:
print("=== Test avec differents parametres ===")

configurations = [
    {"name": "Conservateur", "org": 100, "epochs": 2000, "restarts": 5},
    {"name": "Standard", "org": 200, "epochs": 3000, "restarts": 10},
    {"name": "Agressif", "org": 300, "epochs": 5000, "restarts": 20}
]

test_puzzle = SudokuGrid.from_string(easy_puzzles[0])

for config in configurations:
    solver = PSOSudokuSolver(
        num_organisms=config["org"],
        max_epochs=config["epochs"],
        max_restarts=config["restarts"]
    )
    
    start = time.time()
    result, solved = solver.solve(test_puzzle.clone())
    elapsed = time.time() - start
    
    status = "Succes" if solved else "Echec"
    print(f"{config['name']}: {elapsed:.2f}s - {status}")

=== Test avec differents parametres ===
Tentative 1/5
  Epoch 0, Best error: 23


Conservateur: 0.85s - Succes
Tentative 1/10
  Epoch 0, Best error: 23


Standard: 1.49s - Succes
Tentative 1/20
  Epoch 0, Best error: 21


Agressif: 1.89s - Succes


## 8. Comparaison avec GA et SA

In [11]:
print("=== Comparaison des metaheuristiques ===")
print("\n| Aspect | PSO | Algorithme Genetique | Recuit Simule |")
print("|--------|-----|---------------------|---------------|")
print("| Inspiration | Oiseaux/poissons | Evolution biologique | Thermodynamique |")
print("| Population | Multiple | Multiple | Unique |")
print("| Exploration | Workers + Explorers | Mutation | Temperature |")
print("| Exploitation | Convergence vers gBest | Crossover | Refroidissement |")
print("| Performance Sudoku | ~1-5s | ~1-10s | ~2-5s |")
print("\n**Note**: Ces valeurs sont approximatives et dependent des parametres.")

=== Comparaison des metaheuristiques ===

| Aspect | PSO | Algorithme Genetique | Recuit Simule |
|--------|-----|---------------------|---------------|
| Inspiration | Oiseaux/poissons | Evolution biologique | Thermodynamique |
| Population | Multiple | Multiple | Unique |
| Exploration | Workers + Explorers | Mutation | Temperature |
| Exploitation | Convergence vers gBest | Crossover | Refroidissement |
| Performance Sudoku | ~1-5s | ~1-10s | ~2-5s |

**Note**: Ces valeurs sont approximatives et dependent des parametres.


## 9. Exercices

### Exercice 1 : Analyse de convergence
Modifiez le solveur pour enregistrer l'evolution de `best_error` au fil des epochs et affichez un graphique de convergence.

### Exercice 2 : Adaptation dynamique
Implementez une adaptation dynamique de `WorkerRatio` qui augmente l'exploration quand la solution stagne.

### Exercice 3 : Hybride PSO-SA
Combinez PSO avec le recuit simule : utilisez SA pour affiner les solutions trouvees par PSO.

## Resume

### Concepts cles

| Concept | Description |
|---------|-------------|
| **PSO** | Metaheuristique d'essaim inspiree des oiseaux |
| **Particules** | Solutions candidates avec position et vitesse |
| **pBest/gBest** | Meilleures positions personnelle et globale |
| **Workers** | Exploitent leur voisinage (90% de l'essaim) |
| **Explorers** | Explorent aleatoirement (10% de l'essaim) |

### Parametres cles

| Parametre | Effet | Valeur recommandee |
|-----------|-------|-------------------|
| NumOrganisms | Taille de l'essaim | 100-300 |
| MaxEpochs | Iterations par essai | 3000-5000 |
| MaxRestarts | Redemarrages autorises | 10-20 |
| WorkerRatio | Proportion de workers | 0.85-0.95 |
| MaxAge | Age max avant reinit | 500-1000 |

### Forces et limites

| Avantages | Inconvenients |
|-----------|---------------|
| Parallelisation naturelle | Pas de garantie de convergence |
| Equilibre exploration/exploitation | Performance variable selon puzzles |
| Conceptuellement simple | Parametres a ajuster |

### Quand l'utiliser

- Puzzles moyens (pas trop difficiles)
- Quand on veut plusieurs solutions candidates
- Pour comprendre les metaheuristiques d'essaim

### Alternatives recommandees

- **Backtracking** : [Sudoku-1-Backtracking-Python](Sudoku-1-Backtracking-Python.ipynb)
- **OR-Tools CP-SAT** : [Sudoku-10-ORTools-Python](Sudoku-10-ORTools-Python.ipynb)
- **Z3 SMT** : [Sudoku-12-Z3-Python](Sudoku-12-Z3-Python.ipynb)

---

**Navigation** : [<< Sudoku-4-SimulatedAnnealing](Sudoku-4-SimulatedAnnealing-Python.ipynb) | [Index](README.md) | [Sudoku-6-AIMA-CSP >>](Sudoku-6-AIMA-CSP-Python.ipynb)